In [1]:
%matplotlib inline
import os
import warnings

import critical_loads as cl
import gdal
import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd

warnings.simplefilter("ignore")
plt.style.use("ggplot")

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(database="critical_loads")

Connection successful.


# Critical loads: Water workflow (high-resolution method; 2018 onwards)

In Spring 2018, the worflow for calculating critical loads was refined to make use of new, higher-resolution input datasets. During November 2018, data handling for the Critical Loads project was also redesigned, with the ultimate aim of centralising all key datasets on NIVA's new cloud platform. 

This notebook deals with data processing for the water exceedance calculations, using the new 0.1 degree deposition grid and the old, BLR-based water chemistry dataset. 

## 1. Organising water data on the DSToolkit

NIVA's JupyterHub includes a PostGIS database server capable of storing relational and vector geospatial datasets. I have created a database named `critical_loads` and, within this, a schema named `water`. This schema contains the following tables:

 * **parameter_definitions:** Non-spatial table describing the water chemistry and model parameters used to calculate water critical loads
 
 * **blr_magic_parameters:** Non-spatial table containing result from the MAGIC model
 
 * **blr_optional_parameters:** Non-spatial table containing optional parameters for the calculation of critical loads
 
 * **blr_required_parameters:** Non-spatial table containing required parameters for the calculation of critical loads
 
 * **magic_regression_data:** Output from the MAGIC model used for estimating $BC_0$

## 2. Define deposition data series of interest

Choose the deposition dataset and resolution you wish to work with (see notebook 01 for a list of available `series_ids`). Note that **S and N deposition grids at the specified resolution must already have been created in notebook 02**. 120 m resolution is more than sufficient - the highest resolution of raw data in this workflow is 0.1 degrees, which is several _kilometres_, even in northern Norway.

In [3]:
# Series of interest
ser_id = 28

# Choose cell size (30m, 60m or 120m)
cell_size = 120

## 3. Calculate critical loads for water

The code below reads water chemistry and model parameters for each BLR grid cell from the database and creates rasters for the following key parameters:

    'claoaa', 'eno3', 'clminn', 'clmaxnoaa', 'clmaxsoaa', 'clmins', 'anclimit', 'anclimit_oaa', 'bc0'
    
See the function `docstring` for details - it includes various options for e.g. using Magic data from an Excel template.

#### Added 02.11.2020

The function has been modified to also output `anclimit_ueqpl`, `anclimitoaa_ueqpl` and `bc0_ueqpl`. See e-mail from Kari received 02.11.2020 at 09.30 for details.

In [4]:
# Rasterise critical loads for water
out_fold = r"/home/jovyan/shared/critical_loads/raster/water"
cl.rasterise_water_critical_loads(eng, out_fold, cell_size)

Rasterising claoaa_meqpm2pyr...
Rasterising eno3_flux_meqpm2pyr...
Rasterising clminn_meqpm2pyr...
Rasterising clmaxnoaa_meqpm2pyr...
Rasterising clmaxsoaa_meqpm2pyr...
Rasterising anclimit_ueqpl...
Rasterising anclimitoaa_ueqpl...
Rasterising bc0_ueqpl...
Rasterising clmins_meqpm2pyr...
Rasters saved to:
    /home/jovyan/shared/critical_loads/raster/water


## 4. Calculate exceedances

### 4.1. SSWC model

In [5]:
cl_fold = r"/home/jovyan/shared/critical_loads/raster/water"
out_fold = r"/home/jovyan/shared/critical_loads/raster/exceedance"
ex_df = cl.calculate_water_exceedance_sswc(
    ser_id, "1216", cl_fold, out_fold, cell_size=120, bc0="BC0"
)
ex_df

Exceedance grid saved to:
    /home/jovyan/shared/critical_loads/raster/exceedance/sswc_ex_meqpm2pyr_1216_120m.tif


,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct
0,28,water_sswc,322184,23225,7


**If you connected to the database with `admin=True`**, these results can be saved back to the database.

In [6]:
## Write summary data to db
# ex_df.to_sql('national_summary',
#             eng,
#             'summaries',
#             if_exists='append',
#             index=False)

### 4.2. FAB model

The code in this section calculates exceedances for water using the FAB model. This is based on my original code [here](https://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow_new_grid.ipynb#2.3.3.-FAB-model).

In [7]:
# Constants to build paths
base_path = r"/home/jovyan/shared/critical_loads/raster"
short_name = "1216"

# Read CL arrays
array_dict = {}
for par in [
    "clminn_meqpm2pyr",
    "clmaxnoaa_meqpm2pyr",
    "clmins_meqpm2pyr",
    "clmaxsoaa_meqpm2pyr",
]:
    # Read tif
    tif_path = os.path.join(base_path, "water", f"{par}_{cell_size}m.tif")
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

# Read dep arrays
for par in ["ndep_mgpm2pyr", "sdep_mgpm2pyr"]:
    # Read tif
    tif_path = os.path.join(
        base_path, "deposition", f"{par}_{short_name}_{cell_size}m.tif"
    )
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
    data = data.astype(np.float32)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

**The code below needs to be run on a "high memory" machine, even with 120 m resolution.**

In [8]:
%%time
# Extract arrays from dict
cln_min = array_dict["clminn_meqpm2pyr"]
cln_max = array_dict["clmaxnoaa_meqpm2pyr"]
cls_min = array_dict["clmins_meqpm2pyr"]
cls_max = array_dict["clmaxsoaa_meqpm2pyr"]
dep_n = array_dict["ndep_mgpm2pyr"] / 14  # Convert to meq
dep_s = array_dict["sdep_mgpm2pyr"] * 2 / 32.06  # Convert to meq

# Estimate exceedances
ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(
    cln_min, cln_max, cls_min, cls_max, dep_n, dep_s
)

# Get exceeded area
ex = ex_n + ex_s
ex_area = np.count_nonzero(ex > 0) * cell_size * cell_size / 1.0e6
nor_area = np.count_nonzero(~np.isnan(dep_s)) * cell_size * cell_size / 1.0e6
ex_pct = 100 * ex_area / nor_area

# Build df and tidy
ex_df = pd.DataFrame(
    {
        "exceeded_area_km2": ex_area,
        "total_area_km2": nor_area,
        "exceeded_area_pct": ex_pct,
    },
    index=[0],
)

ex_df = ex_df.round(0).astype(int)
ex_df["series_id"] = ser_id
ex_df["medium"] = "water_fab"

ex_df = ex_df[
    ["series_id", "medium", "total_area_km2", "exceeded_area_km2", "exceeded_area_pct"]
]

ex_df

CPU times: user 10.7 s, sys: 4.96 s, total: 15.7 s
Wall time: 15.7 s


,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct
0,28,water_fab,322184,61056,19


**If you connected to the database with `admin=True`**, these results can be saved back to the database.

In [9]:
## Write summary data to db
# ex_df.to_sql('national_summary',
#             eng,
#             'summaries',
#             if_exists='append',
#             index=False)

The code below writes the FAB results to GeoTiff format.

In [10]:
# Snap tiff
snap_tif = f"/home/jovyan/shared/critical_loads/raster/blr_land_mask_{cell_size}m.tif"

# N
n_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_exn_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n, n_tif, snap_tif, -1, gdal.GDT_Float32)

# S
s_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_exs_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_s, s_tif, snap_tif, -1, gdal.GDT_Float32)

# N+S
ns_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_exns_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n + ex_s, ns_tif, snap_tif, -1, gdal.GDT_Float32)

# Exceedance 'region'
reg_tif = f"/home/jovyan/shared/critical_loads/raster/exceedance/fab_ex_reg_id_{short_name}_{cell_size}m.tif"
cl.write_geotiff(reg_id, reg_tif, snap_tif, -1, gdal.GDT_Float32)